<a href="https://colab.research.google.com/github/higor-gomes93/employee_attrition/blob/main/Employee_Attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Completo - Employee Attrition**

#### **Problema**
Iremos realizar uma análise completa de dados envolvendo o problema de predição de saída de colaboradores em uma empresa. Os dados utilizados podem ser encontrados [aqui](https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset). A anáise será dividida em quatro etapas:
1. Análise básica: conexão com a base de dados, limpeza completa do dataset e análise exploratória gráfica
2. Análise intermediária: correlações, testes de hipóteses e análise de sobrevivência
3. Análise avançada: modelos preditivos, feature importance, oversampling e escolha de um modelo
4. Análise expert: feature selection, grid search e avaliação das métricas de sucesso nas bases de treino e teste com random search

#### **Objetivos**
Identificar quais são as principais alavancas que estão levando as pessoas deixarem a empresa, validar algumas hipóteses e desenvolver um modelo preditivo para essas saídas.




---



## Parte 1 - Análise Básica

Instalação e importação das bibliotecas, conexão com os dados, limpeza do dataset e análise exploratória completa.

### 1) Instalação e Importação das Bibliotecas

Nesta etapa iremos importar as bibliotecas e pacotes necessários para fazer todo o projeto. Conforme visto no módulo de introdução à Python, existem diversas bibliotecas prontas para facilitar o trabalho de análise de dados, e iremos utilizar algumas delas aqui.

In [1]:
# Instalação das bibliotecas
!pip install -q lifelines

     |████████████████████████████████| 349 kB 3.3 MB/s 
     |████████████████████████████████| 68 kB 7.5 MB/s 


In [2]:
# Bibliotecas básicas
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None  # Removendo warnings
import warnings
warnings.filterwarnings('ignore') # Removendo warnings
import numpy as np
import datetime as dt
import statistics as st
import itertools
from scipy import stats

# Bibliotecas de visualização
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as po
from plotly.subplots import make_subplots
import plotly.tools as tls
import plotly.figure_factory as ff

# Importação das bibliotecas de predição
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, r2_score, mean_squared_error, precision_score, recall_score, max_error
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import export_graphviz, DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier

# Imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Kaplan-Meier curve
from lifelines import KaplanMeierFitter

# Bibliotecas adicionais
from dateutil.relativedelta import relativedelta
import graphviz
from sklearn.tree import export_graphviz
from io import StringIO
from IPython.display import Image  
import pydotplus

### 2) Conexão com a Base de Dados

Iremos importar diretamente para nosso código a base de dados dos colaboradores. Ela está localizada em uma pasta no Google Drive, e é possível fazer essa conexão diretamente.

In [3]:
# Pegando o arquivo csv no diretório
path = 'https://raw.githubusercontent.com/higor-gomes93/employee_attrition/main/employee_attrition.csv'
dataset_inicial = pd.read_csv(path)

In [4]:
# Conferindo a conexão
dataset_inicial

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,3,Male,41,4,2,Laboratory Technician,4,Married,2571,12290,4,Y,No,17,3,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,4,Male,42,2,3,Healthcare Representative,1,Married,9991,21457,4,Y,No,15,3,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,2,Male,87,4,2,Manufacturing Director,2,Married,6142,5174,1,Y,Yes,20,4,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,4,Male,63,2,2,Sales Executive,2,Married,5390,13243,2,Y,No,14,3,4,80,0,17,3,2,9,6,0,8


### 3) Observando os Dados

Nessa etapa iremos analisar previamente os dados, observando quais atributos temos disponíveis, o total de observações, algumas medidas estatísticas, tratamentos a serem feitos, o formato e o tipo dos dados, entre outras.

#### 3.1 Observações iniciais

In [5]:
# Observando o formato do dataset inicial
dataset_inicial.head(3)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0


In [6]:
# Observando o total de linhas e colunas
dataset_inicial.shape

(1470, 35)

In [7]:
# Observando o nome das colunas
dataset_inicial.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [8]:
# Observando algumas medidas estatísticas do dataset inicial
dataset_inicial.describe()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,2.912925,1.0,1024.865306,2.721769,65.891156,2.729932,2.063946,2.728571,6502.931293,14313.103401,2.693197,15.209524,3.153741,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.106864,1.024165,0.0,602.024335,1.093082,20.329428,0.711561,1.106940,1.102846,4707.956783,7117.786044,2.498009,3.659938,0.360824,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,1.000000,1.000000,1009.000000,2094.000000,0.000000,11.000000,3.000000,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,491.250000,2.000000,48.000000,2.000000,1.000000,2.000000,2911.000000,8047.000000,1.000000,12.000000,3.000000,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,1020.500000,3.000000,66.000000,3.000000,2.000000,3.000000,4919.000000,14235.500000,2.000000,14.000000,3.000000,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,1.0,1555.750000,4.000000,83.750000,3.000000,3.000000,4.000000,8379.000000,20461.500000,4.000000,18.000000,3.000000,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,5.000000,4.000000,19999.000000,26999.000000,9.000000,25.000000,4.000000,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [9]:
# Observando todas as medidas estatísticas do dataset inicial
dataset_inicial.describe(include = 'all')

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470,1470,1470.000000,1470,1470.000000,1470.000000,1470,1470.0,1470.000000,1470.000000,1470,1470.000000,1470.000000,1470.000000,1470,1470.000000,1470,1470.000000,1470.000000,1470.000000,1470,1470,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
unique,NaN,2,3,NaN,3,NaN,NaN,6,NaN,NaN,NaN,2,NaN,NaN,NaN,9,NaN,3,NaN,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,No,Travel_Rarely,NaN,Research & Development,NaN,NaN,Life Sciences,NaN,NaN,NaN,Male,NaN,NaN,NaN,Sales Executive,NaN,Married,NaN,NaN,NaN,Y,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1233,1043,NaN,961,NaN,NaN,606,NaN,NaN,NaN,882,NaN,NaN,NaN,326,NaN,673,NaN,NaN,NaN,1470,1054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,36.923810,NaN,NaN,802.485714,NaN,9.192517,2.912925,NaN,1.0,1024.865306,2.721769,NaN,65.891156,2.729932,2.063946,NaN,2.728571,NaN,6502.931293,14313.103401,2.693197,NaN,NaN,15.209524,3.153741,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,NaN,NaN,403.509100,NaN,8.106864,1.024165,NaN,0.0,602.024335,1.093082,NaN,20.329428,0.711561,1.106940,NaN,1.102846,NaN,4707.956783,7117.786044,2.498009,NaN,NaN,3.659938,0.360824,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,NaN,NaN,102.000000,NaN,1.000000,1.000000,NaN,1.0,1.000000,1.000000,NaN,30.000000,1.000000,1.000000,NaN,1.000000,NaN,1009.000000,2094.000000,0.000000,NaN,NaN,11.000000,3.000000,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,NaN,NaN,465.000000,NaN,2.000000,2.000000,NaN,1.0,491.250000,2.000000,NaN,48.000000,2.000000,1.000000,NaN,2.000000,NaN,2911.000000,8047.000000,1.000000,NaN,NaN,12.000000,3.000000,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,NaN,NaN,802.000000,NaN,7.000000,3.000000,NaN,1.0,1020.500000,3.000000,NaN,66.000000,3.000000,2.000000,NaN,3.000000,NaN,4919.000000,14235.500000,2.000000,NaN,NaN,14.000000,3.000000,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,NaN,NaN,1157.000000,NaN,14.000000,4.000000,NaN,1.0,1555.750000,4.000000,NaN,83.750000,3.000000,3.000000,NaN,4.000000,NaN,8379.000000,20461.500000,4.000000,NaN,NaN,18.000000,3.000000,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000


In [10]:
# Algumas informações adicionais
dataset_inicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

#### 3.2) Observando valores nulos

In [11]:
# Identificando valores nulos
dataset_inicial.isnull().any()

Age                         False
Attrition                   False
BusinessTravel              False
DailyRate                   False
Department                  False
DistanceFromHome            False
Education                   False
EducationField              False
EmployeeCount               False
EmployeeNumber              False
EnvironmentSatisfaction     False
Gender                      False
HourlyRate                  False
JobInvolvement              False
JobLevel                    False
JobRole                     False
JobSatisfaction             False
MaritalStatus               False
MonthlyIncome               False
MonthlyRate                 False
NumCompaniesWorked          False
Over18                      False
OverTime                    False
PercentSalaryHike           False
PerformanceRating           False
RelationshipSatisfaction    False
StandardHours               False
StockOptionLevel            False
TotalWorkingYears           False
TrainingTimesL

#### 3.3) Observando outliers

In [12]:
# Criando dicionários para adicioinar os valores
dict_outliers_zscore = {}
dict_outliers_iqr = {}
dict_outliers_percentile = {}

In [13]:
# Definindo uma função para coleta de outliers por meio do método Z-score com maior robustez
def z_r_score_outlier(data):
    outliers = []
    med = np.median(data)
    ma = stats.median_absolute_deviation(data)
    for i in data: 
        z = (0.6745*(i-med))/ (np.median(ma))
        if np.abs(z) > 3: 
            outliers.append(i)
    return outliers

# Definindo uma função para coleta de outliers por meio do método IQR
def iqr_outliers(data):
    outliers = []
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3-q1
    Lower_tail = q1 - 2.5 * iqr
    Upper_tail = q3 + 2.5 * iqr
    for i in data:
        if i > Upper_tail or i < Lower_tail:
            outliers.append(i)
    return outliers

# Definindo uma função para coleta de outliers por meio da análise do percentil
def winsorization_outliers(data):
    outliers = []
    q1 = np.percentile(data, 1)
    q3 = np.percentile(data, 99)
    for i in data:
        if i > q3 or i < q1:
            outliers.append(i)
    return outliers

In [14]:
# Coletando os outliers
for i in dataset_inicial.columns:
    if dataset_inicial[i].dtype == 'int64':
        outliers_zscore = z_r_score_outlier(dataset_inicial[i])
        if len(outliers_zscore) > 0:
            dict_outliers_zscore[i] = outliers_zscore
        outliers_iqr = iqr_outliers(dataset_inicial[i])
        if len(outliers_iqr) > 0:
            dict_outliers_iqr[i] = outliers_iqr
        outliers_percentile = winsorization_outliers(dataset_inicial[i])
        if len(outliers_percentile) > 0:
            dict_outliers_percentile[i] = outliers_percentile

### 4) Limpeza e Tratamento dos Dados

Etapa de remoção de outliers, padronização de categorias, tratamento de valores faltantes e outras transformações necessárias.

#### 4.1) Removendo colunas sem valor semântico

In [15]:
# Colunas sem importância para as análises
dataset_inicial = dataset_inicial.drop(columns = ['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'])

#### 4.2) Valores faltantes

In [16]:
# Conforme indicado anteriormente, não existem valores nulos na base de dados
print('Não existem valores nulos.') if any(list(dataset_inicial.isnull().any())) == False else print('Existem valores nulos.')

Não existem valores nulos.


#### 4.3) Outliers e valores fora do padrão

In [17]:
# Iremos nos basear na técnica que analise o percentil dos dados
# Coletando os outliers
dict_outliers_percentile = {}
for i in dataset_inicial.columns:
    if dataset_inicial[i].dtype == 'int64':
        outliers_percentile = winsorization_outliers(dataset_inicial[i])
        if len(outliers_percentile) > 0:
            dict_outliers_percentile[i] = outliers_percentile

# Observando os outliers
for i in dict_outliers_percentile.items():
    print(i)

('Age', [59, 59, 59, 59, 59, 59, 18, 18, 60, 60, 18, 60, 18, 59, 59, 18, 60, 59, 59, 18, 18, 60, 18])
('DailyRate', [103, 1488, 111, 1496, 111, 106, 1490, 1490, 1499, 1495, 102, 109, 1492, 111, 116, 107, 1498, 1495, 1490, 1496, 115, 104, 1495, 1490, 116, 105])
('MonthlyIncome', [1232, 19926, 1102, 19999, 1200, 1009, 1281, 19859, 1051, 19973, 19845, 1052, 19627, 19943, 19740, 1223, 1118, 19847, 19717, 19701, 1359, 1261, 1274, 19658, 19833, 19665, 1081, 1091, 19636, 1129])
('MonthlyRate', [2094, 26959, 26897, 26820, 2302, 2137, 26767, 26707, 26914, 2227, 2288, 2112, 2125, 26894, 26999, 2104, 2243, 26968, 2253, 26933, 2323, 2261, 2097, 26997, 26841, 2125, 2122, 26862, 26849, 26956])
('TotalWorkingYears', [0, 37, 38, 40, 0, 36, 37, 0, 0, 36, 37, 0, 40, 0, 0, 0, 36, 36, 0, 36, 0, 36, 37, 0])
('YearsAtCompany', [37, 40, 33, 33, 36, 32, 34, 32, 33, 33, 32, 33, 36])
('YearsInCurrentRole', [16, 18, 18, 17, 16, 16, 16, 16, 16, 17, 17, 17, 16])
('YearsSinceLastPromotion', [15, 15, 15, 15, 15, 15,

In [18]:
'''
Observando os valores de outliers obtidos, será tomada a decisão de não eliminar nenhum 
registro (ou substituí-lo), pois eles têm um significado no contexto de predição de saídas.
'''

'\nObservando os valores de outliers obtidos, será tomada a decisão de não eliminar nenhum \nregistro (ou substituí-lo), pois eles têm um significado no contexto de predição de saídas.\n'

#### 4.4) Variáveis categóricas fora do padrão

In [19]:
# Selecionando as variáveis categóricas
variaveis_categoricas = dataset_inicial.select_dtypes(exclude = ["int64"])

# Observando se existem valores despadronizados
for i in variaveis_categoricas:
    aux = dataset_inicial.groupby([i]).count().reset_index().iloc[:, 0:2]
    aux.columns = [i, 'Contagem']
    display(aux)
    print("\n\n")

,Attrition,Contagem
0,No,1233
1,Yes,237


,BusinessTravel,Contagem
0,Non-Travel,150
1,Travel_Frequently,277
2,Travel_Rarely,1043


,Department,Contagem
0,Human Resources,63
1,Research & Development,961
2,Sales,446


,EducationField,Contagem
0,Human Resources,27
1,Life Sciences,606
2,Marketing,159
3,Medical,464
4,Other,82
5,Technical Degree,132


,Gender,Contagem
0,Female,588
1,Male,882


,JobRole,Contagem
0,Healthcare Representative,131
1,Human Resources,52
2,Laboratory Technician,259
3,Manager,102
4,Manufacturing Director,145
5,Research Director,80
6,Research Scientist,292
7,Sales Executive,326
8,Sales Representative,83


,MaritalStatus,Contagem
0,Divorced,327
1,Married,673
2,Single,470


,OverTime,Contagem
0,No,1054
1,Yes,416


#### 4.5) Criando uma cópia

In [20]:
dataset = dataset_inicial.copy()

### 5) Análise Exploratória do Dados

Iremos observar o dataset de forma gráfica, a fim de identificar alguns pontos que possam servir para análises futuras, além de entender as características e comportamentos da base.

#### 5.1) Distribuição da variável alvo

In [21]:
# Criando dois datasets parciais para ativos e inativos
dataset_inativos = dataset[(dataset['Attrition'] == 'Yes')]
dataset_ativos = dataset[(dataset['Attrition'] == 'No')]

# Porcentagem
percent_lista = [f'{round(len(dataset_inativos)/len(dataset), 3)*10*10}%', f'{round(len(dataset_ativos)/len(dataset), 3)*10*10}%']

# Criando um visual de barras
trace_1 = go.Bar(x = (len(dataset_inativos), len(dataset_ativos)), 
                 y = ['Inativos', 'Ativos'], 
                 orientation = 'h', 
                 opacity = 0.8,
                 text = percent_lista,
                 marker = dict(color = ['lightsalmon', 'lightgreen'], line = dict(color = '#000000', width = 1.5)))

# Plotando os gráficos
fig_1 = go.Figure(trace_1)
fig_1.update_layout(width = 1000, height = 500, title = "Contagem da variável alvo", plot_bgcolor = "white", title_x = 0.5)
fig_1.update_yaxes(showline = True, linewidth = 1, linecolor = 'black')
fig_1.update_xaxes(showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE')
fig_1.show()

#### 5.2) Variáveis numéricas - distribuições

In [22]:
# Selecionando as variáveis numéricas de interesse (sem as categóricas)
colunas_dist_plot_num = dataset.nunique()[dataset.nunique() > 10].keys().tolist()
colunas_dist_plot_num = [x for x in colunas_dist_plot_num]

# Definindo um limite superior para a plotagem
limite_superior = int(len(colunas_dist_plot_num)/2) if len(colunas_dist_plot_num)/2 == len(colunas_dist_plot_num)//2 else len(colunas_dist_plot_num)//2 + 1

# Plotando todos os gráficos
fig = make_subplots(rows = limite_superior, cols = 2, subplot_titles = colunas_dist_plot_num)
contador = 0

for i in range(1, limite_superior+1):
    for j in range(1, 3):
        try:
            # Legenda
            legenda = True if contador == 0 else False

            # Criando os datasets auxiliares
            aux_1 = dataset_inativos[colunas_dist_plot_num[contador]]
            aux_2 = dataset_ativos[colunas_dist_plot_num[contador]]
            
            # Criando os atributos do gráfico
            data = [aux_1, aux_2]
            labels = ['Inativos', 'Ativos']
            colors = ['#FFA07A', '#90EE90']

            # Obtendo os dados 
            fig1 = ff.create_distplot(hist_data = data, group_labels = labels, colors = colors, show_rug = False, bin_size = False)

            # Criando o gráfico
            fig.add_trace(go.Histogram(fig1['data'][0], 
                                       marker_color = colors[0], 
                                       showlegend = legenda, 
                                       opacity = 0.6), 
                          row = i, col = j)
            fig.add_trace(go.Histogram(fig1['data'][1], 
                                       marker_color = colors[1], 
                                       showlegend = legenda, 
                                       opacity = 0.6), 
                          row = i, col = j)
            fig.add_trace(go.Scatter(fig1['data'][2], 
                                     line = dict(color = colors[0], width = 1.5)), 
                          row = i, col = j)
            fig.add_trace(go.Scatter(fig1['data'][3], 
                                     line = dict(color = colors[1], width = 1.5)),
                           row = i, col = j)
            contador += 1
        except:
            pass

fig.update_layout(width = 1500, height = 2000, title = "Distribuição das variáveis numéricas de interesse", plot_bgcolor = "white", title_x = 0.5)
fig.update_yaxes(showline = True, linewidth = 1, linecolor = 'black', rangemode = 'tozero')
fig.update_xaxes(showline = True, linewidth = 1, linecolor = 'black', showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE', rangemode = 'nonnegative')
fig.show()

#### 5.3) Variáveis numéricas - barras

In [23]:
# Selecionando as variáveis numéricas de interesse (sem as categóricas)
colunas_bar_plot_num = dataset.nunique()[(dataset.nunique() < 50) & (dataset.nunique() > 10)].keys().tolist()
colunas_bar_plot_num = [x for x in colunas_bar_plot_num]

# Definindo um limite superior para a plotagem
limite_superior = int(len(colunas_bar_plot_num)/2) if len(colunas_bar_plot_num)/2 == len(colunas_bar_plot_num)//2 else len(colunas_bar_plot_num)//2 + 1

# Plotando todos os gráficos
fig = make_subplots(rows = limite_superior, cols = 2, subplot_titles = colunas_bar_plot_num, specs = [[{'secondary_y': True}, {'secondary_y': True}]]*limite_superior)
contador = 0

for i in range(1, limite_superior+1):
    for j in range(1, 3):
        try:
            # Legenda
            legenda = True if contador == 0 else False

            # Criando os datasets auxiliares
            aux_1 = dataset_inativos
            aux_2 = dataset_ativos
            aux_3 = pd.DataFrame(pd.crosstab(dataset[colunas_bar_plot_num[contador]], dataset['Attrition']), )
            aux_3['% de Saídas'] = aux_3['Yes'] / (aux_3['Yes'] + aux_3['No']) * 100
            aux_3.columns = ['No', 'Yes', '% de Saídas']
            aux_3.index.name = None
            
            # Criando os atributos do gráfico
            data = [aux_1, aux_2]
            labels = ['Inativos', 'Ativos']
            colors = ['#FFA07A', '#90EE90']

            # Criando o gráfico
            fig.add_trace(go.Bar(x = aux_1[colunas_bar_plot_num[contador]].value_counts().keys().tolist(), 
                                    y = aux_1[colunas_bar_plot_num[contador]].value_counts().values.tolist(), 
                                    name = 'Inativos', 
                                    opacity = 0.8,
                                    showlegend = legenda,
                                    marker = dict(color = 'LightSalmon', line = dict(color = '#000000', width = 1))), 
                            secondary_y = False,
                            row = i, col = j)
            fig.add_trace(go.Bar(x = aux_2[colunas_bar_plot_num[contador]].value_counts().keys().tolist(), 
                                    y = aux_2[colunas_bar_plot_num[contador]].value_counts().values.tolist(), 
                                    name = 'Ativos', 
                                    opacity = 0.8,
                                    showlegend = legenda, 
                                    marker = dict(color = 'LightGreen', line = dict(color = '#000000', width = 1))), 
                            secondary_y = False,
                            row = i, col = j)
            fig.add_trace(go.Scatter(x = aux_3.index, 
                                        y = aux_3['% de Saídas'], 
                                        name = '% de Saídas', 
                                        opacity = 0.6, 
                                        showlegend = legenda,
                                        marker = dict(color = 'black', line = dict(color = '#000000', width = 0.5))), 
                            secondary_y = True,
                            row = i, col = j)
            contador += 1
        except:
            pass

fig.update_layout(width = 1500, height = 1500, title = "Barras das variáveis numéricas de interesse", plot_bgcolor = "white", title_x = 0.5)
fig.update_yaxes(showline = False, linewidth = 1, linecolor = 'black', rangemode = 'tozero', secondary_y = False, title = "Contagem")
fig.update_yaxes(showline = False, linewidth = 1, linecolor = 'black', rangemode = 'tozero', secondary_y = True, title = "% de Saídas", range = [-0, 100])
fig.update_xaxes(showline = True, linewidth = 1, linecolor = 'black', showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE', rangemode = 'nonnegative')
fig.show()

#### 5.4) Variáveis categóricas - barras

In [24]:
# Selecionando as variáveis numéricas de interesse (categóricas)
colunas_bar_plot_cat = dataset.nunique()[dataset.nunique() < 10].keys().tolist()
colunas_bar_plot_cat = [x for x in colunas_bar_plot_cat]
colunas_bar_plot_cat.remove('Attrition')

# Definindo um limite superior para a plotagem
limite_superior = int(len(colunas_bar_plot_cat)/2) if len(colunas_bar_plot_cat)/2 == len(colunas_bar_plot_cat)//2 else len(colunas_bar_plot_cat)//2 + 1

# Plotando todos os gráficos
fig = make_subplots(rows = limite_superior, cols = 2, subplot_titles = colunas_bar_plot_cat, specs = [[{'secondary_y': True}, {'secondary_y': True}]]*limite_superior)
contador = 0

for i in range(1, limite_superior+1):
    for j in range(1, 3):
        try:
            # Legenda
            legenda = True if contador == 0 else False

            # Criando os datasets auxiliares
            aux_1 = dataset_inativos
            aux_2 = dataset_ativos
            aux_3 = pd.DataFrame(pd.crosstab(dataset[colunas_bar_plot_cat[contador]], dataset['Attrition']), )
            aux_3['% de Saídas'] = aux_3['Yes'] / (aux_3['Yes'] + aux_3['No']) * 100
            aux_3.columns = ['No', 'Yes', '% de Saídas']
            aux_3.index.name = None
            if isinstance(aux_3.index[0], str):
                aux_3 = aux_3.sort_values(by = 'Yes', ascending = False)
            
            # Criando os atributos do gráfico
            data = [aux_1, aux_2]
            labels = ['Inativos', 'Ativos']
            colors = ['#FFA07A', '#90EE90']

            # Criando o gráfico
            fig.add_trace(go.Bar(x = aux_1[colunas_bar_plot_cat[contador]].value_counts().keys().tolist(), 
                                    y = aux_1[colunas_bar_plot_cat[contador]].value_counts().values.tolist(), 
                                    name = 'Inativos', 
                                    opacity = 0.8,
                                    showlegend = legenda,
                                    marker = dict(color = 'LightSalmon', line = dict(color = '#000000', width = 1))), 
                            secondary_y = False,
                            row = i, col = j)
            fig.add_trace(go.Bar(x = aux_2[colunas_bar_plot_cat[contador]].value_counts().keys().tolist(), 
                                    y = aux_2[colunas_bar_plot_cat[contador]].value_counts().values.tolist(), 
                                    name = 'Ativos', 
                                    opacity = 0.8,
                                    showlegend = legenda, 
                                    marker = dict(color = 'LightGreen', line = dict(color = '#000000', width = 1))), 
                            secondary_y = False,
                            row = i, col = j)
            fig.add_trace(go.Scatter(x = aux_3.index, 
                                        y = aux_3['% de Saídas'], 
                                        name = '% de Saídas', 
                                        opacity = 0.6, 
                                        showlegend = legenda,
                                        marker = dict(color = 'black', line = dict(color = '#000000', width = 0.5))), 
                            secondary_y = True,
                            row = i, col = j)
            contador += 1
        except:
            pass

fig.update_layout(width = 1500, height = 3000, title = "Barras das variáveis numéricas de interesse", plot_bgcolor = "white", title_x = 0.5)
fig.update_yaxes(showline = False, linewidth = 1, linecolor = 'black', rangemode = 'tozero', secondary_y = False, title = "Contagem")
fig.update_yaxes(showline = False, linewidth = 1, linecolor = 'black', rangemode = 'tozero', secondary_y = True, title = "% de Saídas", range = [-0, 100])
fig.update_xaxes(showline = True, linewidth = 1, linecolor = 'black', showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE', rangemode = 'nonnegative')
fig.show()

#### 5.5) Matrix de scatter plot

In [25]:
# Selecionando as variáveis numéricas
colunas_scatter = colunas_bar_plot_cat.copy()
colunas_scatter.append("Attrition")

# Criando um dataset numérico
dataset_numerico_aux = dataset.drop(colunas_scatter, axis = 1)

# Criando um dataset apenas com a variável attrition
dataset_attrition = dataset[['Attrition']]

# Unindo os datasets (para alterar a ordem)
dataset_numerico = pd.concat([dataset_attrition, dataset_numerico_aux], axis = 1)

In [26]:
# Criando o visual para todas as variáveis numéricas
fig = px.scatter_matrix(dataset_numerico, 
                        dimensions = list(dataset_numerico.columns)[1:-1], 
                        color = list(dataset_numerico.columns)[0],
                        color_discrete_sequence = ['#FFA07A', '#90EE90'],
                        labels = ['Inativos', 'Ativos']
                       )

# Atualizando o layout geral
fig.update_layout(width = 2500, height = 3000, title = "Scatterplot Matrix com todas as variáveis numéricas", title_x = 0.5)
fig.update_traces(opacity = 0.75)
fig.show()

In [27]:
# Selecionando as variáveis numéricas com maior variabilidade
colunas_cont_num = dataset.nunique()[dataset.nunique() > 30].keys().tolist()
colunas_cont_num = [x for x in colunas_cont_num]

# Criando um dataset apenas com as variáveis de interesse
colunas_scatter_2 = colunas_cont_num.copy()

# Criando um dataset numérico
dataset_numerico_2_aux = dataset[colunas_scatter_2]

# Unindo os datasets (para alterar a ordem)
dataset_numerico_2 = pd.concat([dataset_attrition, dataset_numerico_2_aux], axis = 1)

In [28]:
# Criando o visual para todas as variáveis numéricas
fig = px.scatter_matrix(dataset_numerico_2, 
                        dimensions = list(dataset_numerico_2.columns)[1:-1], 
                        color = list(dataset_numerico_2.columns)[0],
                        color_discrete_sequence = ['#FFA07A', '#90EE90'],
                        labels = ['Inativos', 'Ativos']
                       )

# Atualizando o layout geral
fig.update_layout(width = 1500, height = 1500, title = "Scatterplot Matrix com todas as variáveis numéricas contínuas", title_x = 0.5)
fig.update_traces(opacity = 0.75)
fig.show()

## Parte 2 - Análise Intermediária

Correlaçãoes, testes de hipóteses e análise de sobrevivência.

### 6) Tratamento Adicional ao Dataset

Iremos aqui realizar o processo de transformação das variáveis categóricas e também a normalização dos dados.

#### 6.1) Codificação das features com Label Encoder

In [29]:
# Criando uma cópia do dataset original para mantê-lo
dataset_label_encoder = dataset.copy()

In [30]:
# Selecionando as features categóricas
features_categoricas = dataset_label_encoder[colunas_bar_plot_cat].select_dtypes(exclude = ["int64"]).keys().tolist()

# Criando um objeto codificador
le = LabelEncoder()

# Codificando as colunas
for i in features_categoricas:
    dataset_label_encoder[i] = le.fit_transform(dataset_label_encoder[i])

In [31]:
# Alterando a ordem da variável alvo
dataset_label_encoder = dataset_label_encoder.drop(['Attrition'], axis = 1)
dataset_label_encoder = pd.concat([dataset_attrition, dataset_label_encoder], axis = 1)

# Codificando a variável alvo
dataset_label_encoder['Attrition'] = dataset_label_encoder['Attrition'].apply(lambda x: 1 if x == "Yes" else 0)

# Conferindo o tipo das colunas
set(list(dataset_label_encoder.dtypes))

{dtype('int64')}

#### 6.2) Utilizando a técnica de variáveis dummy para codificar

In [32]:
# Criando uma cópia para facilitar o manueio
dataset_dummy = dataset.copy()

In [33]:
# Variável alvo
target_col = ["Attrition"]

# Features categóricas
cat_cols   = dataset_dummy.nunique()[dataset_dummy.nunique() < 10].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]

# Features numéricas
num_cols   = [x for x in dataset_dummy.columns if x not in cat_cols + target_col]

# Features binárias
bin_cols   = dataset_dummy.nunique()[dataset_dummy.nunique() == 2].keys().tolist()

# Colunas com mais de dois valores
multi_cols = [i for i in cat_cols if i not in bin_cols]

In [34]:
# Utilizando Label Encoding apenas para as variáveis binárias (0 e 1)
le = LabelEncoder()
for i in bin_cols :
    dataset_dummy[i] = le.fit_transform(dataset_dummy[i])
    
# Utilizando a técnica de variáveis dummy para features com mais de dois valores
dataset_dummy = pd.get_dummies(data = dataset_dummy, columns = multi_cols)

#### 6.3) Normalização das features

In [35]:
# Normalizando entre 0 e 1 o dataset_label_encoder
min_max_scaler = MinMaxScaler()
normalizadas = min_max_scaler.fit_transform(dataset_label_encoder)
dataset_label_encoder_norm = pd.DataFrame(normalizadas, columns = dataset_label_encoder.columns)

In [36]:
# Normalizando o dataset_dummy
std = StandardScaler()
scaled = std.fit_transform(dataset_dummy[num_cols])
dataset_dummy_scaled_aux = pd.DataFrame(scaled, columns = num_cols)

#dropping original values merging scaled values for numerical columns
dataset_dummy_original = dataset_dummy.copy()
dataset_dummy_scaled = dataset_dummy.drop(columns = num_cols, axis = 1)
dataset_dummy_scaled = dataset_dummy_scaled.merge(dataset_dummy_scaled_aux, left_index = True, right_index = True,how = "left")

### 7) Correlação

Etapa dedicada à plotagem da matriz de correlação, assim como a análise de variáveis colineares.

#### 7.1) Matrizes de correlação

In [56]:
# Obtendo as correlações
correl_data = dataset_label_encoder.corr()

# Obtendo as colunas
colunas_correl = correl_data.columns.tolist()

# Convertendo para um vetor
correl_array  = np.array(correl_data)

# Criando um objeto de figura
fig = go.Figure()

# Criando o plot
trace = go.Heatmap(z = correl_array,
                   x = colunas_correl,
                   y = colunas_correl,
                   colorscale = 'rdylgn',
                  )

# Adicionando
fig.add_trace(trace)

# Atualizando o layout
fig.update_layout(width = 800, height = 800, title = "Matriz de Correlação", title_x = 0.5)
fig.update_traces(opacity = 0.75)
fig.show()

In [57]:
# Obtendo as correlações para ativos e inativos
correl_data_ativos = dataset_label_encoder.loc[dataset_label_encoder['Attrition'] == 0].iloc[:, 1:-1].corr()
correl_data_inativos = dataset_label_encoder.loc[dataset_label_encoder['Attrition'] == 1].iloc[:, 1:-1].corr()

# Obtendo as colunas
colunas_correl_ativos = correl_data_ativos.columns.tolist()
colunas_correl_inativos = correl_data_inativos.columns.tolist()

# Convertendo para um vetor
correl_array_ativos  = np.array(correl_data_ativos)
correl_array_inativos  = np.array(correl_data_inativos)

# Criando um objeto de figura
fig = make_subplots(rows = 1, cols = 2, subplot_titles = ('Matriz de Correlações - Ativos', 'Matriz de Correlações - Inativos'))

# Criando os plots
trace_1 = go.Heatmap(z = correl_array_ativos,
                   x = colunas_correl_ativos,
                   y = colunas_correl_ativos,
                   colorscale = 'rdylgn',
                  )

trace_2 = go.Heatmap(z = correl_array_inativos,
                   x = colunas_correl_inativos,
                   y = colunas_correl_inativos,
                   colorscale = 'rdylgn',
                  )

# Adicionando
fig.append_trace(trace_1, row = 1, col = 1)
fig.append_trace(trace_2, row = 1, col = 2)

# Atualizando o layout
fig.update_layout(width = 1000, height = 500, title = "Matriz de Correlação - Comparativo", title_x = 0.5)
fig.update_yaxes(visible = False)
fig.update_xaxes(visible = False)
fig.update_traces(opacity = 0.75)
fig.show()

#### 7.2) Remoção de variáveis com alta correlação

In [59]:
# Definindo um limite máximo de correlação permitido
limite = 0.8

# Obtendo os valores absolutos de correlação
matriz_correl_abs = dataset_label_encoder.corr().abs()
matriz_correl_abs.head()

# Removendo os valores abaixo da diagonal principal da matriz (incluindo a diagonal principal)
matriz_correl_abs_filtrada = matriz_correl_abs.where(np.triu(np.ones(matriz_correl_abs.shape), k=1).astype(np.bool))
matriz_correl_abs_filtrada.head()

# Identificando as colunas a serem removidas
features_colineares = [feature for feature in matriz_correl_abs_filtrada.columns if any(matriz_correl_abs_filtrada[feature] > limite)]

In [60]:
# Observando as features colineares
features_colineares

['MonthlyIncome']

In [61]:
# Removendo a coluna
dataset_label_encoder = dataset_label_encoder.drop(features_colineares, axis = 1)
dataset_label_encoder_norm = dataset_label_encoder_norm.drop(features_colineares, axis = 1)
dataset_dummy = dataset_dummy.drop(features_colineares, axis = 1)
dataset_dummy_scaled = dataset_dummy_scaled.drop(features_colineares, axis = 1)